## Vectorized vs Devectorized

In [1]:
const n = 1000
const k = 1000

1000

In [2]:
const a = rand(n,1);
const b = rand(n,1);
const c = rand(n,1);

In [3]:
function find_delta_devectorized()
    positive_delta = []

    for i in 1:n
        @inbounds @fastmath d = b[i]^2-4a[i]c[i]
        if d.>0
            push!(positive_delta,[a[i], b[i], c[i]])
        end
    end

    return positive_delta
end

function find_delta_vectorized()
    alldelta = b.^2-4*a.*c
    return [a b c][alldelta.>0,:]
end

@time for i in 1:k find_delta_devectorized() end
@time for i in 1:k find_delta_vectorized() end

  0.049202 seconds (1.09 M allocations: 45.334 MB, 14.12% gc time)
  0.252658 seconds (106.59 k allocations: 70.529 MB, 7.00% gc time)


In [4]:
# from http://stackoverflow.com/questions/27928502/julia-vectorized-vs-devectorized-code
const x = rand(n)
const y = rand(n)
const u = rand()
const v = rand()

0.8802404795496619

In [5]:
function devect3()
    pairs = []

    for i in 1:n
        @inbounds @fastmath dist = sqrt((x[i] - u)^2 + (y[i] - v)^2)
        if dist < 0.05
            push!(pairs,[x[i], y[i]])
        end
    end

    return pairs
end

function devect2()
    pairs = Array(Float64, n, 2)

    for i in 1:n
        dist = sqrt((x[i] - u)^2 + (y[i] - v)^2)
        if dist < 0.05
            pairs[i,:] = [x[i], y[i]]
        end
    end

    return pairs
end

function vect()
    d = sqrt((x-u).^2 + (y-v).^2)
    return [x y][d .< 0.05,:]
end

@time for i in 1:n devect3() end
@time for i in 1:n devect2() end
@time for i in 1:n vect() end

  0.011662 seconds (27.00 k allocations: 1.038 MB)
  0.014225 seconds (31.00 k allocations: 16.281 MB, 27.60% gc time)
  0.142840 seconds (19.00 k allocations: 66.254 MB, 7.05% gc time)


In [6]:
const P1 = rand(n,2);
const P2 = rand(n,2);

In [7]:
function find_slopes_devec()
    small_slopes = []

    for i in 1:n
        @inbounds @fastmath slope = (P2[i,2] - P1[i,2])/(P2[i,1] - P1[i,1])
        if slope < 0.01
            push!(small_slopes,i)
        end
    end

    return small_slopes
end

function find_slopes_vectorized()
    allslopes = (P2[:,2]-P1[:,2])./((P2[:,1] - P1[:,1]))
    return allslopes.<0.01
end

@time for i in 1:k find_slopes_devec() end
@time for i in 1:k find_slopes_vectorized() end

  0.029779 seconds (256.00 k allocations: 19.714 MB, 10.78% gc time)
  0.079799 seconds (55.95 k allocations: 59.705 MB, 15.65% gc time)


`const` can change value but cannot change type

Why not global variables? Julia recommends avoiding global variables specially when performance is important
>A global variable might have its value, and therefore its type, change at any point. This makes it difficult for the compiler to optimize code using global variables. Variables should be local, or passed as arguments to functions, whenever possible.

## Iterators
For a more extensive study on iterators in Julia, check this [post](http://slendermeans.org/julia-iterators.html)

This is very similar to Python's `__iter__` and `__next__`. We will see next Julia's `start`, `next` and `done` -- An important difference here is that these functions **do not** have any effect on the object being iterated.

In [8]:
I = [1,11,21,31,41,51];
@show start(I);
@show next(I,1);
@show done(I,2);

start(I) = 1
next(I,1) = (1,2)
done(I,2) = false


In [9]:
@show state = start(I);
@show i,state = next(I,state);
@show i,state = next(I,state);
@show i,state = next(I,state);
@show i,state = next(I,state);

state = start(I) = 1
(i,state) = next(I,state) = (1,2)
(i,state) = next(I,state) = (11,3)
(i,state) = next(I,state) = (21,4)
(i,state) = next(I,state) = (31,5)


In [10]:
state = start(I)
while !done(I,state)
    i, state = next(I,state)
    println("I am the element number $(state-1) and my value is $i")
end

I am the element number 1 and my value is 1
I am the element number 2 and my value is 11
I am the element number 3 and my value is 21
I am the element number 4 and my value is 31
I am the element number 5 and my value is 41
I am the element number 6 and my value is 51


In [11]:
# iterators can run on 2D arrays columnwise
J = hcat(I,I)
state = start(J)
while !done(J,state)
    i, state = next(J,state)
    println("I am the element number $(state-1) and my value is $i")
end

I am the element number 1 and my value is 1
I am the element number 2 and my value is 11
I am the element number 3 and my value is 21
I am the element number 4 and my value is 31
I am the element number 5 and my value is 41
I am the element number 6 and my value is 51
I am the element number 7 and my value is 1
I am the element number 8 and my value is 11
I am the element number 9 and my value is 21
I am the element number 10 and my value is 31
I am the element number 11 and my value is 41
I am the element number 12 and my value is 51


In [12]:
# iterators on dictionaries:
D = Dict(:score1 => 100, :score2 => 70, :score3 => 99, :score4 => 80)

Dict{Symbol,Int64} with 4 entries:
  :score1 => 100
  :score4 => 80
  :score3 => 99
  :score2 => 70

In [13]:
# Dictionaries are a little different.
@show state = start(D);
@show i,state = next(D,state);
@show i,state = next(D,state);
@show i,state = next(D,state);
@show i,state = next(D,state);

state = start(D) = 1
(i,state) = next(D,state) = (:score1=>100,5)
(i,state) = next(D,state) = (:score4=>80,14)
(i,state) = next(D,state) = (:score3=>99,15)
(i,state) = next(D,state) = (:score2=>70,17)


In [14]:
D.keys

16-element Array{Symbol,1}:
    :score1
 #undef    
 #undef    
 #undef    
    :score4
 #undef    
 #undef    
 #undef    
 #undef    
 #undef    
 #undef    
 #undef    
 #undef    
    :score3
    :score2
 #undef    

In [15]:
# if an iterator makes sense on a type that you've created, then you can define your own iterator

type Houses
    beds::Array{Int64,1}
    baths::Array{Int64,1}
    sq__ft::Array{Float64,1}
end

Base.start(h::Houses) = 1
Base.next(h::Houses,state::Int64) = ([h.beds[state],h.baths[state],h.sq__ft[state]],state+1)
Base.done(h::Houses,state::Int64) = length(h.beds) == state-1

h = Houses([3,3,2],[2,1,1],[98937,68212,59222])

for i in h
    println("$i")
end

[3.0,2.0,98937.0]
[3.0,1.0,68212.0]
[2.0,1.0,59222.0]


## Column major storage of matrices

In [30]:
function square_rowmajor(A)
    m,n = size(A)
    for i=1:m, j=1:n
        #fix row (i), change col (j)
        A[i,j] += 1
    end
end

function square_colmajor(A)
    m,n = size(A)
    for j=1:n, i=1:m
        #fix col (j), change row (i)
        A[i,j] += 1
    end
end

# sz = 100
sz = 5000
A = rand(sz,sz);
@time square_colmajor(A)
@time square_rowmajor(A)

  0.000029 seconds (7 allocations: 288 bytes)
  0.000056 seconds (7 allocations: 288 bytes)


## Inplace operations

In [45]:
function square_inplace!{T}(v::Vector{T})
    for i = 1:length(v)
        v[i] = v[i]^2
    end
    v
end

function square_not_inplace{T}(v::Vector{T})
    w = zeros(eltype(v),length(v))
    for i = 1:length(v)
        w[i] = v[i]^2
    end
    w
end

# function square_not_inplace(v)

square_not_inplace (generic function with 1 method)

In [54]:
vec = [1;2;3]
@time square_not_inplace(vec)

  0.000008 seconds (5 allocations: 256 bytes)


3-element Array{Int64,1}:
 1
 4
 9

In [55]:
vec

3-element Array{Int64,1}:
 1
 2
 3

In [56]:
vec = [1,2,3]
@time square_inplace!(vec)

  0.000004 seconds (4 allocations: 160 bytes)


3-element Array{Int64,1}:
 1
 4
 9

In [57]:
vec

3-element Array{Int64,1}:
 1
 4
 9

## Careful wit types @code_warntype

In [1]:
function my_sum(vec)
    s = 0
    for i = 1:length(vec)
        s += vec[i]
    end
    return s
end

my_sum (generic function with 1 method)

In [2]:
vec = rand(1000000);

In [3]:
@show my_sum(vec)
@show sum(vec)

my_sum(a) = 500512.6108968449
sum(a) = 500512.6108968367


500512.6108968367

In [56]:
@time my_sum(vec)

  0.041744 seconds (2.00 M allocations: 30.518 MB, 10.26% gc time)


500512.6108968449

In [57]:
@code_warntype my_sum(vec)

Variables:
  vec::Array{Float64,1}
  s::ANY
  #s52::Int64
  i::Int64

Body:
  begin  # In[1], line 2:
      s = 0 # In[1], line 3:
      GenSym(2) = (Base.arraylen)(vec::Array{Float64,1})::Int64
      GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,GenSym(2))::Bool,GenSym(2),(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64)))
      #s52 = (top(getfield))(GenSym(0),:start)::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s52::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Bool)) goto 1
      2: 
      GenSym(3) = #s52::Int64
      GenSym(4) = (Base.box)(Base.Int,(Base.add_int)(#s52::Int64,1))
      i = GenSym(3)
      #s52 = GenSym(4) # In[1], line 4:
      s = s::UNION{FLOAT64,INT64} + (Base.arrayref)(vec::Array{Float64,1},i::Int64)::Float64::Float64
      3: 
      unless (Base.box)(Base.Bool,(Base.not_int)((Base.box)(Base.Bool,(Base.not_int)(#s52::Int64 === (Base.box)(

In [67]:
function my_sum_faster(vec)
    s = zero(eltype(vec))
    for i = 1:length(vec)
        s += vec[i]
    end
    return s
end

my_sum_faster (generic function with 1 method)

In [68]:
@code_warntype my_sum_faster(vec)

Variables:
  vec::Array{Float64,1}
  s::Float64
  #s52::Int64
  i::Int64

Body:
  begin  # In[67], line 2:
      s = (Base.box)(Float64,(Base.sitofp)(Float64,0)) # In[67], line 3:
      $(Expr(:boundscheck, false))
      GenSym(2) = (Base.arraylen)(vec::Array{Float64,1})::Int64
      GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,GenSym(2))::Bool,GenSym(2),(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64)))
      #s52 = (top(getfield))(GenSym(0),:start)::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s52::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Bool)) goto 1
      2: 
      GenSym(3) = #s52::Int64
      GenSym(4) = (Base.box)(Base.Int,(Base.add_int)(#s52::Int64,1))
      i = GenSym(3)
      #s52 = GenSym(4) # In[67], line 4:
      s = (Base.box)(Base.Float64,(Base.add_float)(s::Float64,(Base.arrayref)(vec::Array{Float64,1},i::Int64)::Float64))
      3: 
      un

In [72]:
@time my_sum_faster(vec)
@time my_sum(vec)

  0.001644 seconds (5 allocations: 176 bytes)
  0.043430 seconds (2.00 M allocations: 30.518 MB, 4.15% gc time)


500512.6108968449

## @simd and @inbounds

## Type Stability

In [217]:
function square_positive(val)
    if val>=0
        return val^2
    else
        return -1
    end
end

@show square_positive(rand())
@show square_positive(-3.2)

@show typeof(square_positive(rand()))
@show typeof(square_positive(-3.2))

square_positive(rand()) = 0.9269297164744749
square_positive(-3.2) = -1
typeof(square_positive(rand())) = Float64
typeof(square_positive(-3.2)) = Int64


Int64

In [218]:
@show square_positive(9)
@show square_positive(-3)

@show typeof(square_positive(9))
@show typeof(square_positive(-3))

square_positive(9) = 81
square_positive(-3) = -1
typeof(square_positive(9)) = Int64
typeof(square_positive(-3)) = Int64


Int64

In [219]:
# A type-stable function can let the compiler predict the type of the output based on the type of the input
function square_positive_stable(val)
    if val>=0
        return val^2
    else
        return one(typeof(val))*-1
    end
end

square_positive_stable (generic function with 1 method)

In [221]:
@show square_positive_stable(rand())
@show square_positive_stable(-3.2)

@show typeof(square_positive_stable(rand()))
@show typeof(square_positive_stable(-3.2))

square_positive_stable(rand()) = 0.031372300638581985
square_positive_stable(-3.2) = -1.0
typeof(square_positive_stable(rand())) = Float64
typeof(square_positive_stable(-3.2)) = Float64


Float64

In [222]:
@show square_positive(9)
@show square_positive(-3)

@show typeof(square_positive(9))
@show typeof(square_positive(-3))

square_positive(9) = 81
square_positive(-3) = -1
typeof(square_positive(9)) = Int64
typeof(square_positive(-3)) = Int64


Int64

## More on types

In [223]:
# code from: https://en.wikibooks.org/wiki/Introducing_Julia/Types
level = 0
function showtypetree(subtype)
    global level
    subtypelist = filter(asubtype -> asubtype != Any, subtypes(subtype))
    if length(subtypelist) > 0 
         println("\t" ^ level, subtype)        
         level += 1
         map(showtypetree, subtypelist)
         level -= 1
    else
         println("\t" ^ level, subtype)
    end    
end

showtypetree(Number)

Number
	Complex{T<:Real}
	Real
		AbstractFloat
			BigFloat
			Float16
			Float32
			Float64
		Integer
			BigInt
			Bool
			Signed
				Int128
				Int16
				Int32
				Int64
				Int8
			Unsigned
				UInt128
				UInt16
				UInt32
				UInt64
				UInt8
		Irrational{sym}
		Rational{T<:Integer}


0

## Multiple Dispatch

In [225]:
foo(x::ASCIIString)  = println("The type of your input is String")
foo(x::Integer) = println("The type of your input is Integer")
foo(x::Float64) = println("The type of your input is Float")
methods(foo)

# 3 methods for generic function "foo":
foo(x::ASCIIString) at In[225]:1
foo(x::Integer) at In[225]:2
foo(x::Float64) at In[225]:3

## Functional Programming

In [238]:
funcmap = x -> x^2
@show map(funcmap, 2)
@show map(*,["Hello","Hi"],[" World.", "there!"])

map(funcmap,2) = 4
map(*,["Hello","Hi"],[" World.","there!"]) = ASCIIString["Hello World.","Hithere!"]


2-element Array{ASCIIString,1}:
 "Hello World."
 "Hithere!"    

## Metaprogramming


In [240]:
prog = "1 + 1"

"1 + 1"

In [242]:
ex1 = parse(prog)

:(1 + 1)

In [243]:
typeof(ex1)

Expr

In [244]:
ex1.args

3-element Array{Any,1}:
  :+
 1  
 1  

In [290]:
function functional_mymult(x,y)
    mx,nx = size(x)
    my,ny = size(y)
    assert(mx==my)
    assert(nx==ny)
    
    for i = 1:length(x)
        x_i = x[i]
        y_i = y[i]
        exp = :($x_i * $y_i)
        print(join([exp, " = "]))
        println(eval(exp))
    end
end

vec1 = rand(3,3)
vec2 = rand(3,3)
functional_mymult(vec1,vec2)

0.00030976955923511795 * 0.5771688163806854 = 0.00017878932985449964
0.11705519000844977 * 0.5744767792688561 = 0.0672454885527582
0.9895346528592499 * 0.2422579730275114 = 0.23972265924216402
0.49989503157206094 * 0.7600013039791633 = 0.37992087584747136
0.12561768614811486 * 0.460822003356387 = 0.05788739378776816
0.2706035947381029 * 0.28522620300569246 = 0.07718323584684027
0.16496090410303177 * 0.7359209712067707 = 0.1213981887586501
0.7858765755310992 * 0.34838814669529805 = 0.2737900836805271
0.5478768814910593 * 0.6585847006673498 = 0.36082333199935035
